In [2]:
# Import Library
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend
import ipywidgets as widgets
from IPython.display import display

# AES Functions
def derive_key(password):
    return password.encode().ljust(32, b"0")[:32]

def aes_encrypt(data, password):
    key = derive_key(password)
    iv = os.urandom(16)

    padder = padding.PKCS7(128).padder()
    padded_data = padder.update(data) + padder.finalize()

    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    encrypted = encryptor.update(padded_data) + encryptor.finalize()

    return iv + encrypted

def aes_decrypt(data, password):
    key = derive_key(password)
    iv = data[:16]
    encrypted = data[16:]

    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    decrypted_padded = decryptor.update(encrypted) + decryptor.finalize()

    unpadder = padding.PKCS7(128).unpadder()
    decrypted = unpadder.update(decrypted_padded) + unpadder.finalize()

    return decrypted

# Output Folder
OUTPUT_FOLDER = "output"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Widgets
file_uploader = widgets.FileUpload(accept='', multiple=False)
mode_input = widgets.Text(description="Mode (E/D):")
password_input = widgets.Password(description="Password:")
run_button = widgets.Button(description="Process", button_style='success')
output_area = widgets.Output()

display(file_uploader, mode_input, password_input, run_button, output_area)

# Process Logic
def process_action(b):
    with output_area:
        output_area.clear_output()

        if len(file_uploader.value) == 0:
            print("❌ Tidak ada file yang dipilih.")
            return

        mode = mode_input.value.lower()
        if mode not in ['e','d']:
            print("❌ Mode harus 'E' (enkripsi) atau 'D' (dekripsi).")
            return

        password = password_input.value
        if password.strip() == "":
            print("❌ Password tidak boleh kosong.")
            return

        # ambil file
        file_info = next(iter(file_uploader.value.values()))
        filename = file_info['metadata']['name']
        file_bytes = file_info['content']

        print("📁 File:", filename)
        print("🔐 Mode:", "Encrypt" if mode=='e' else "Decrypt")

        if mode == 'e':
            result = aes_encrypt(file_bytes, password)
            out_path = os.path.join(OUTPUT_FOLDER, filename + ".enc")
        else:
            try:
                result = aes_decrypt(file_bytes, password)
            except:
                print("❌ Password salah atau file corrupt.")
                return
            out_path = os.path.join(OUTPUT_FOLDER, filename.replace(".enc",""))

        # simpan file output
        with open(out_path, "wb") as f:
            f.write(result)

        print("✅ Selesai!")
        print("📦 Output disimpan di:", out_path)

run_button.on_click(process_action)


FileUpload(value=(), description='Upload')

Text(value='', description='Mode (E/D):')

Password(description='Password:')

Button(button_style='success', description='Process', style=ButtonStyle())

Output()